In [4]:
import gc
import argparse
import pandas as pd
import numpy as np
import re
from scipy import sparse
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
import fasttext
import fasttext.util
from transliterate import translit

Собственный vectorizer для входных данных

In [7]:
# 1) Читаем TSV
df = pd.read_csv("text_data_new.tsv", sep='\t')

In [8]:
df

,index,word_count
0,81619,сериал 10 series 2 ivi 2 бесплатно 2 serialov ...
1,288108,mini 2 prokopevsk 2 sistema 2 стеллажи 2 вход ...
2,252671,uslug 1 костроме 2 doctors 1 работу 2 подслуша...
3,123476,мессенджер 4 услуги 1 ильи 2 объявления 1 доба...
4,13202,сериал 3 века 1 информация 2 внешний 1 феррари...
...,...,...
274837,24979,логинов 1 shojgu 1 shampuni 1 годы 1 косметики...
274838,336266,норма 1 сиквел 1 eda 1 zdorovya 1 иномарки 1 п...
274839,343454,экспорт 3 опубликовало 6 жестокость 3 продлить...
274840,123422,мобильные 3 снять 2 метки 1 самовывозом 2 mnp ...


In [9]:
df.dropna(inplace=True)

В текстах много слов транслитов -> нужно обработать

In [11]:
df

,index,word_count
0,81619,сериал 10 series 2 ivi 2 бесплатно 2 serialov ...
1,288108,mini 2 prokopevsk 2 sistema 2 стеллажи 2 вход ...
2,252671,uslug 1 костроме 2 doctors 1 работу 2 подслуша...
3,123476,мессенджер 4 услуги 1 ильи 2 объявления 1 доба...
4,13202,сериал 3 века 1 информация 2 внешний 1 феррари...
...,...,...
274837,24979,логинов 1 shojgu 1 shampuni 1 годы 1 косметики...
274838,336266,норма 1 сиквел 1 eda 1 zdorovya 1 иномарки 1 п...
274839,343454,экспорт 3 опубликовало 6 жестокость 3 продлить...
274840,123422,мобильные 3 снять 2 метки 1 самовывозом 2 mnp ...


In [13]:
from transliterate import detect_language, translit
from pymorphy3 import MorphAnalyzer
import re

morph_ru = MorphAnalyzer(lang='ru')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ivanp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer

eng_stem = PorterStemmer()
rus_stem = SnowballStemmer("russian")
rus_sw = set(stopwords.words('russian'))
eng_sw = set(stopwords.words('english'))

In [17]:
def is_russian_word(cyrillic_word):
    threshold = 0.6
    parsed = morph_ru.parse(cyrillic_word)
    if parsed[0].score > threshold:
        return True
    return False

In [18]:
import re
from functools import lru_cache
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer
from transliterate import translit
from pymorphy3 import MorphAnalyzer

# Предкомпилируем паттерн
_re_russian = re.compile('[а-яА-Я]')

# Инициализируем стеммеры и стоп‑слова один раз
rus_stem = SnowballStemmer("russian")
eng_stem = PorterStemmer()
rus_sw = set(stopwords.words('russian'))
eng_sw = set(stopwords.words('english'))
morph_ru = MorphAnalyzer(lang='ru')

def _is_russian_word(word: str, threshold: float = 0.6) -> bool:
    parsed = morph_ru.parse(word)
    return parsed and parsed[0].score > threshold

@lru_cache(maxsize=None)
def _stem_token(token: str) -> str:
    # Пропускаем стоп‑слова
    if token in rus_sw or token in eng_sw:
        return None

    # Если есть кириллица — сразу русское стеммирование
    if _re_russian.search(token):
        return rus_stem.stem(token)

    # Иначе пробуем транслит и проверку через pymorphy
    conv = translit(token, 'ru')
    if _is_russian_word(conv):
        return rus_stem.stem(conv)
    else:
        return eng_stem.stem(token)

def stem(text: str) -> str:
    """
    Вход: строка вида "word1 count1 word2 count2 ..."
    Выход: строка, где каждое stemmed слово повторяется count раз.
    """
    parts = text.split()
    out_tokens = []

    # Обрабатываем парами (токен, count)
    for tok, cnt_str in zip(parts[0::2], parts[1::2]):
        cnt = int(cnt_str)
        st = _stem_token(tok)
        if st:
            # добавляем st повторённый cnt раз
            out_tokens.append(f"{st} {cnt}")

    return ' '.join(out_tokens)


In [19]:
from tqdm import tqdm
tqdm.pandas()  # активация прогресс-бара
df['word_count'] = df['word_count'].progress_apply(stem)

100%|██████████| 274446/274446 [06:22<00:00, 716.70it/s] 


In [29]:
import joblib
joblib.dump(df, 'ptext_v1.joblib')

['ptext_v1.joblib']